In [1]:
!pip install transformers
!pip install datasets
!pip list
!pip install rouge_score
!pip install bert_score

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Package                  Version            
------------------------ -------------------
absl-py                  2.1.0              
accelerate               0.27.2             
aiohttp                  3.9.3              
aiosignal                1.3.1              
alembic                  1.4.3              
appdirs                  1.4.4              
argon2-cffi              20.1.0             
async-generator          1.10               
async-timeout            4.0.3              
attrs                    20.3.0             
backcall                 0.2.0              
batchspawner             1.0.1              
bert-score               0.3.13             
bleach                   3.2.1              
blinker                  1.4                
certifi                  2020.12.5          
certipy                  0.1.3    

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
from datasets import load_dataset

/home/csci5541/mohan130/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# By Madhan Mohan

# Task 1

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

prompts = [
    "With a heavy heart,",
    "Standing on the edge,",
    "Surrounded by strangers,",
    "Caught in a dilemma,",
    "At the crossroads,"
]






In [4]:
num_beams = 5


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i, prompt in enumerate(prompts):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    beam_search_config = model.generate(
        input_ids, 
        max_length=30, 
        num_beams=num_beams,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    
    output = tokenizer.batch_decode(beam_search_config, skip_special_tokens=True)
    
    encodings = tokenizer(output, return_tensors="pt").to(device)
    print("Output:", output)
    print()
    
    # Attribution of code: https://huggingface.co/docs/transformers/main/en/perplexity
    max_length = model.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        encodings = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = encodings.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(encodings, labels=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    print("Generated Text:", output)
    print("Perplexity: ", ppl)



Output: ["With a heavy heart, I've decided to take a look at some of the most important things you can do to make your life easier.\n\n"]



  0%|          | 0/1 [00:00<?, ?it/s]

Generated Text: ["With a heavy heart, I've decided to take a look at some of the most important things you can do to make your life easier.\n\n"]
Perplexity:  tensor(5.9687)


Output: ['Standing on the edge, he said, "I don\'t know what to do. I don\'t know what to do. I don\'t know what']



  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Generated Text: ["Caught in a dilemma, I'm going to try to find a way to make it work for me.\n\nI'm going to try to"]
Perplexity:  tensor(7.3199)


Output: ['At the crossroads, he said, "I don\'t know what I\'m going to do. I don\'t know what I\'m going to do']



  0%|          | 0/1 [00:00<?, ?it/s]

Generated Text: ['At the crossroads, he said, "I don\'t know what I\'m going to do. I don\'t know what I\'m going to do']
Perplexity:  tensor(3.7023)


In [5]:
# Task 2

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

dataset = load_dataset('cnn_dailymail', '1.0.0')

test_dataset = dataset["test"]

sentence_list = []
reference_list = []

counter = 0
for item in test_dataset:
    if counter == 50:
        break
    
    sentence_list.append(item["article"])
    reference_list.append(item["highlights"])
    counter += 1



prediction_list = []
for i in range(len(sentence_list)):
    input_ids = tokenizer(sentence_list[i], return_tensors="pt", max_length=1024, truncation=True).input_ids
    res = model.generate(input_ids, num_beams=5, do_sample=False, max_length=30)
    prediction_list.append(tokenizer.batch_decode(res, skip_special_tokens=True)[0])




/home/csci5541/mohan130/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1202: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (30). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


In [6]:
print("Input Text")
for i in range(len(sentence_list)):
    print(sentence_list[i][:1024])
    print()

Input Text
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's c

In [7]:
print("Reference Text")
for i in range(len(sentence_list)):
    print(reference_list[i])
    print()

Reference Text
Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June . Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

Theia, a bully breed mix, was apparently hit by a car, whacked with a hammer and buried in a field . "She's a true miracle dog and she deserves a good life," says Sara Mellado, who is looking for a home for Theia .

Mohammad Javad Zarif has spent more time with John Kerry than any other foreign minister . He once participated in a takeover of the Iranian Consulate in San Francisco . The Iranian foreign minister tweets in English .

17 Americans were exposed to the Ebola virus while in Sierra Leone in March . Another person was diagnosed with the disease and taken to hospital in Maryland . National Institutes of Health says the patient is in fair condition after weeks of treatment .

Student is no longer on Duke University campus and will fa

In [9]:
# Task 3

import evaluate

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=prediction_list, references=reference_list, use_aggregator=False)
print("Rouge Results: ", results)




Rouge Results:  {'rouge1': [0.44827586206896547, 0.46875, 0.48275862068965525, 0.3692307692307692, 0.44776119402985076, 0.22222222222222224, 0.1842105263157895, 0.27692307692307694, 0.44897959183673475, 0.46511627906976744, 0.490566037735849, 0.20000000000000004, 0.24489795918367346, 0.14705882352941177, 0.2631578947368421, 0.12307692307692307, 0.4482758620689655, 0.26666666666666666, 0.5652173913043478, 0.30188679245283023, 0.2105263157894737, 0.21874999999999997, 0.3692307692307692, 0.4210526315789474, 0.5660377358490566, 0.2028985507246377, 0.3111111111111111, 0.19047619047619047, 0.20833333333333331, 0.4782608695652174, 0.30379746835443033, 0.18181818181818182, 0.45, 0.20833333333333331, 0.4482758620689655, 0.3333333333333333, 0.3508771929824562, 0.34374999999999994, 0.2711864406779661, 0.18181818181818185, 0.6222222222222222, 0.1095890410958904, 0.6956521739130435, 0.5333333333333333, 0.5714285714285715, 0.13043478260869565, 0.8799999999999999, 0.22641509433962265, 0.2307692307692

In [ ]:
bert = evaluate.load("bertscore")
results = rouge.compute(predictions=prediction_list, references=reference_list, lang="en")
print("Bert Results: ", results)